![image.png](attachment:image.png)

## PRACTICA OBLIGATORIA: **Transfer Learning y Fine Tuning**

* La práctica obligatoria de esta unidad consiste en un repetir el ejercicio de construcción de un modelo de clasificación de paisajes pero usando un modelo preentrenado. Descarga este notebook en tu ordenador y trabaja en local. Ten en cuenta que tendrás que descar los directorios de imágenes y datos adicionales, si los hubiera.
* Recuerda que debes subirla a tu repositorio personal antes de la sesión en vivo para que puntúe adecuadamente.  
* Recuerda también que no es necesario que esté perfecta, sólo es necesario que se vea el esfuerzo.
* Esta práctica se resolverá en la sesión en vivo correspondiente y la solución se publicará en el repo del curso.

### Ejercicio 0

Importa los paquetes y módulos que necesites a lo largo del notebook.

In [11]:
import os
import cv2
import numpy as np
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Flatten
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.metrics import classification_report, confusion_matrix
from tensorflow.keras.utils import to_categorical
from sklearn.preprocessing import LabelEncoder
from skimage.io import imread
!pip install gdown
!pip install rarfile


### Objetivo del ejercicio

Comparar una red convolucional hecha ad-hoc frente a los modelos preentrenados y ajustados con fine tuning y transfer learning. Para ello emplea el dataset de paisajes del conjunto de ejercicios de la unidad anterior.


### Se pide

1. Preparar los datos del modelo y las funciones de visualización, copia para ello todo lo que necesites de las soluciones del ejercicio de clasificación de paisajes de la unidad anterior.

2. Escoger entre uno de los modelos VGG-19, InceptionV3 y MobileNetV2 (todos en https://keras.io/api/applications/) (Se aconseja este último si no tenemos un ordenador muy potente). Si no te haces con estos puedes recurrir a la ResNetV50.

4. Hacer un transfer-learning con una cabeza de como mucho 2 capas densas ocultas y una de salida. Mostrar la evaluación contra test, el report de clasificación y la matriz de confusión.

5. Hacer un fine-tuning con la misma cabeza diseñada en el punto anterior. Mostrar la evaluación contra test, el report de clasificación y la matriz de confusión.

6. Comparar los resultados con los obtenidos con la red convolucional del ejercicio mencionado.

EXTRA:
- Repetir el transfer learning empleando aumentado de imágenes.



In [15]:
!pip install rarfile

!apt-get install unrar

import gdown
import rarfile
import pandas as pd

# Paso 1: Descarga el archivo .rar desde Google Drive
file_id = '1WAd1uCRsb479rqAiy85yhMyM5iuQ-UHf'  # Reemplaza con el ID de tu archivo
gdown.download(f'https://drive.google.com/drive/folders/1WAd1uCRsb479rqAiy85yhMyM5iuQ-UHf', 'data.rar', quiet=False)

# Paso 2: Extrae el archivo CSV directamente desde el .rar
with rarfile.RarFile('data.rar', 'r') as rar:
    with rar.open('mnist_clase_convo.rar') as file:  # Cambia al nombre correcto de tu archivo .csv dentro del .rar
        df = pd.read_csv(file)

# Mostrar las primeras filas del DataFrame
print(df.head())


Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
unrar is already the newest version (1:6.1.5-1).
0 upgraded, 0 newly installed, 0 to remove and 49 not upgraded.


Downloading...
From: https://drive.google.com/drive/folders/1WAd1uCRsb479rqAiy85yhMyM5iuQ-UHf
To: /content/data.rar
1.19MB [00:00, 53.3MB/s]


NotRarFile: Not a RAR file

In [7]:
df = pd.read_csv("./data/mnist_clase_convo.csv")

FileNotFoundError: [Errno 2] No such file or directory: './data/mnist_clase_convo.csv'

In [5]:
df.info()

NameError: name 'df' is not defined

In [6]:
df.describe

NameError: name 'df' is not defined

In [16]:
df.head()

NameError: name 'df' is not defined

In [17]:
ROOT_PATH = './data/'
TRAIN_PATH = ROOT_PATH + "seg_train/"
TEST_PATH = ROOT_PATH + "seg_test/"

In [ ]:
def read_data(directorio, reshape_dim=(160, 160)):
    X = []
    y = []
    for folder in os.listdir(directorio):
        print(folder)
        if os.path.isdir(os.path.join(directorio, folder)):
            for file in os.listdir(os.path.join(directorio, folder)):

                image_path = os.path.join(directorio, folder, file)
                image = imread(image_path)
                image_resized = cv2.resize(image, reshape_dim) # Redimensionamos las imágenes a 160x160

                X.append(image_resized)
                y.append(folder)

    return np.array(X), np.array(y)


In [ ]:
X_train,y_train = read_data(TRAIN_PATH)
X_test, y_test = read_data(TEST_PATH)

In [ ]:
# String a int
label_encoder = LabelEncoder()
y_train_encoded = label_encoder.fit_transform(y_train)
y_test_encoded = label_encoder.transform(y_test)

y_train_one_hot = to_categorical(y_train_encoded, num_classes=6)
y_test_one_hot = to_categorical(y_test_encoded, num_classes=6)

In [ ]:
print('X train shape: ', X_train.shape)
print('Y train shape: ', y_train.shape)

In [ ]:
print('X train shape: ', X_test.shape)
print('Y train shape: ', y_test.shape)

In [ ]:
def show_images_batch(paisajes, names = [], n_cols = 5, size_scale = 2):
    n_rows = ((len(paisajes) - 1) // n_cols + 1)
    plt.figure(figsize=(n_cols * size_scale, n_rows * 1.1*size_scale))
    for index, paisaje in enumerate(paisajes):
        plt.subplot(n_rows, n_cols, index + 1)
        plt.imshow(paisaje, cmap = "Greys")
        plt.axis("off")
        if len(names):
            plt.title(names[index])

In [ ]:
indices = np.random.randint(0,len(X_train),40)
show_images_batch(X_train[indices], names = y_train[indices], n_cols = 5)